The goal of this notebook is to take the ad hoc implementation in `MIPExample1.ipynb` and vectorize the operations such that any size Tyche problem can be converted to a MIP and solved.

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath("../src"))

In [2]:
import numpy             as np
import matplotlib.pyplot as pl
import pandas            as pd
import seaborn           as sb
import tyche             as ty

from copy            import deepcopy
from IPython.display import Image 

In [3]:
import cProfile
import timeit

In [4]:
from mip import Model, MINIMIZE, MAXIMIZE, BINARY, xsum
from itertools import product

In [5]:
designs = ty.Designs("data")
investments = ty.Investments("data")
designs.compile()
tranche_results = investments.evaluate_tranches(designs, sample_count=250)
results = investments.tranches.join(tranche_results.summary)
evaluator = ty.Evaluator(investments.tranches, tranche_results.summary)

Get the wide-format interpolated elicitation data from the Tyche Evaluator and reset the multi-level index.

In [6]:
wide = evaluator.evaluate_corners_wide().reset_index()

In [7]:
wide.head(9)

Index,CIGS,CdTe,GaAs,InGaP,Perovskite,Polysilicon,Power Electronics,Soft Costs,Capital,Efficiency,GHG,Hazardous,LCOE,Lifetime,Strategic,Yield
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.540736,2.066534,-0.003592,0.975012,-0.144656,188.002094,0.06354,9994.488309
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1000000.0,-1.457593,2.066306,-0.003592,0.975275,-0.142055,188.002155,0.06354,9994.488166
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5000000.0,-1.287314,2.066872,-0.003592,0.974622,-0.136727,188.002058,0.06354,9994.491669
3,0.0,0.0,0.0,0.0,0.0,0.0,1000000.0,0.0,-1.469833,2.066220,-0.003592,0.963551,-0.138740,188.002163,0.06354,10058.378702
4,0.0,0.0,0.0,0.0,0.0,0.0,1000000.0,1000000.0,-1.386691,2.065993,-0.003592,0.963813,-0.136138,188.002223,0.06354,10058.378558
5,0.0,0.0,0.0,0.0,0.0,0.0,1000000.0,5000000.0,-1.216412,2.066558,-0.003592,0.963160,-0.130811,188.002126,0.06354,10058.382062
6,0.0,0.0,0.0,0.0,0.0,0.0,5000000.0,0.0,-1.373995,2.068111,-0.003592,0.954248,-0.133654,188.002106,0.06354,10101.813236
7,0.0,0.0,0.0,0.0,0.0,0.0,5000000.0,1000000.0,-1.290852,2.067883,-0.003592,0.954510,-0.131052,188.002167,0.06354,10101.813093
8,0.0,0.0,0.0,0.0,0.0,0.0,5000000.0,5000000.0,-1.120573,2.068449,-0.003592,0.953858,-0.125724,188.002070,0.06354,10101.816596


**Input to MIP constructor needed**: List of investment categories

**Input to MIP constructor needed**: List of metrics to optimize and/or constrain

**Data check**: Confirm that all elements of both input lists match columns in the `evaluate_corners_wide()` data frame.

In [8]:
categories = ['Polysilicon', 'Power Electronics', 'Soft Costs']

metrics = ['Capital']

Pull out the investment values and metric values from the data set.

In [9]:
# Investment levels
inv_levels = wide.loc[:26,categories].values.tolist()

# Elicited metric values
m = wide.loc[:26,metrics].values.tolist()

# Number of endpoints
I = len(inv_levels)

Instantiate the MIP optimization problem.

In [10]:
example = Model(sense=MAXIMIZE)

Create binary (integer) variables $y_{in_i}$.

In [11]:
bin_vars = []

for i in range(I-1):
    _name = 'y' + '_' + str(i)
    bin_vars += [example.add_var(name=_name, var_type=BINARY)]
    del _name

Create continuous $\lambda$ variables with lower bound 0.0 and upper bound 1.0

In [12]:
lmbd_vars = []

for i in range(I):
    _name = 'lmbd' + '_' + str(i)
    lmbd_vars += [example.add_var(name=_name, lb=0.0, ub=1.0)]
    del _name

Define upper bound for total budget constraint and per-category upper bounds.

In [13]:
B = 3000000.0
max_amount = [1000000.0, 2000000.0, 1500000.0]

Create total budget constraint as a function of the $\lambda$ variables and the elicited investment levels from the data set.

In [14]:
example += xsum(lmbd_vars[i] * inv_levels[i][j] 
                for i in range(I)
                for j in range(len(inv_levels[i]))) <= B, 'Total Budget'

Create constraints for budget within each investment category.

In [15]:
for j in range(len(categories)):
    example += xsum(lmbd_vars[i] * [el[j] for el in inv_levels][i] 
                    for i in range(I)) <= max_amount[j], 'Budget for ' + categories[j]

Create three sets of constraints:
* Convexity constraints on $\lambda$ variables: the $\lambda$s must sum to 1.
* Constrain binary $y$ variables within each category such that the decision variables lie within exactly one interval (also a sum-to-1 constraint).
* Interval constraints on $y$ variables and $\lambda$ variables. There will be as many interval constraints per investment category as there are linear intervals.

In [16]:
example += sum(lmbd_vars) == 1, 'Lambda Sum'
example += sum(bin_vars) == 1, 'Binary Sum'

In [17]:
for i in range(I-1):
    example += bin_vars[i] <= lmbd_vars[i] + lmbd_vars[i+1], 'Interval Constraint' + str(i)

Create objective function: Capital (metric) as a function of $\lambda$s and $y$s.

Create a data frame of $\lambda$ variables by mapping every element in `v` to its corresponding $\lambda$, using column name (investment category) and investment level.

In [18]:
example.objective = xsum(m[i][0] * lmbd_vars[i] for i in range(I))

Save model to a `lp` file for manual examination.

In [19]:
example.write('example.lp')

Optimize.

In [20]:
example.optimize()

<OptimizationStatus.OPTIMAL: 0>

Print optimal objective function value, with a negative applied to reverse the -1.0 in the objective function definition (minimizing a negative -> maximize a positive).

In [21]:
example.objective_value

-1.3654058993756168

Get the optimal $\lambda$ and $y$ values. Store as separate lists and also print out.

In [31]:
lmbd_opt = []
y_opt = []

for v in example.vars:
    if 'lmbd' in v.name:
        lmbd_opt += [v.x]
    elif 'y' in v.name:
        y_opt += [v.x]
    print('{} : {}'.format(v.name, v.x))

y_0 : 0.0
y_1 : 0.0
y_2 : 0.0
y_3 : 0.0
y_4 : 1.0
y_5 : 0.0
y_6 : 0.0
y_7 : 0.0
y_8 : 0.0
y_9 : 0.0
y_10 : 0.0
y_11 : 0.0
y_12 : 0.0
y_13 : 0.0
y_14 : 0.0
y_15 : 0.0
y_16 : 0.0
y_17 : 0.0
y_18 : 0.0
y_19 : 0.0
y_20 : 0.0
y_21 : 0.0
y_22 : 0.0
y_23 : 0.0
y_24 : 0.0
y_25 : 0.0
lmbd_0 : 0.0
lmbd_1 : 0.0
lmbd_2 : 0.0
lmbd_3 : 0.0
lmbd_4 : 0.875
lmbd_5 : 0.125
lmbd_6 : 0.0
lmbd_7 : 0.0
lmbd_8 : 0.0
lmbd_9 : 0.0
lmbd_10 : 0.0
lmbd_11 : 0.0
lmbd_12 : 0.0
lmbd_13 : 0.0
lmbd_14 : 0.0
lmbd_15 : 0.0
lmbd_16 : 0.0
lmbd_17 : 0.0
lmbd_18 : 0.0
lmbd_19 : 0.0
lmbd_20 : 0.0
lmbd_21 : 0.0
lmbd_22 : 0.0
lmbd_23 : 0.0
lmbd_24 : 0.0
lmbd_25 : 0.0
lmbd_26 : 6.577832330824488e-17


Calculate the optimal investment levels.

In [25]:
inv_levels_opt = {}

for i in range(len(categories)):
    inv_levels_opt[categories[i]] = sum([lmbd_opt[j] * [el[i] for el in inv_levels][j] 
                                         for j in range(len(lmbd_opt))])

inv_levels_opt

{'Polysilicon': 2.96002454887102e-10,
 'Power Electronics': 1000000.0000000003,
 'Soft Costs': 1500000.0000000002}

Calculate the optimal total investment.

In [30]:
total_inv_levels_opt = sum(inv_levels_opt.values())
total_inv_levels_opt

2500000.000000001